In [2]:
import argparse

import pandas


from preprocess import preprocess_list, write_pkl
# from rnn_w2v import run_testing, run_training, RNN_Model
# from word_embedder_gensim import WordVectorizer

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [8]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Ana\AppData\Roaming\nltk_data...


True

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ana\AppData\Roaming\nltk_data...


True

In [3]:
import pickle

In [4]:
dbtrain = pandas.read_csv("dbpedia_csv/" + "train.csv", names=["label", "title", "content"])
# dbvalid = pandas.read_csv(paths.data + "validation.csv", names=["label", "title", "content"])
dbtest = pandas.read_csv("dbpedia_csv/" + "test.csv", names=["label", "title", "content"])


train_docs = list(dbtrain["content"])
#valid_docs = list(dbvalid["content"])
test_docs = list(dbtest["content"])

In [9]:

new_train_docs = preprocess_list(train_docs)
write_pkl(list(dbtrain["label"]), list(dbtrain["title"]), new_train_docs, mode="train_lem_")

Preprocessing: 100%|█████████████████████████████████████████████████████████| 560000/560000 [07:54<00:00, 1180.26it/s]


In [10]:
new_test_docs = preprocess_list(test_docs)
write_pkl(list(dbtest["label"]), list(dbtest["title"]), new_test_docs, mode="test_lem_")

Preprocessing: 100%|███████████████████████████████████████████████████████████| 70000/70000 [00:51<00:00, 1359.18it/s]


## tf-idf naive bayes

In [24]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

'prophecy fifth novel new york times bestselling author chris kuzneski published october 2009 penguin uk action thriller follows adventures jonathon payne david dj jones try decipher newly discovered manuscript written nostradamus book peaked 4 british fiction list stayed bestseller list several weeks putnam released american hardcover version july 2010'

In [14]:
y_train, x_b, x_train = pickle.load(open("dbpedia_csv/" + "train_lem__preprocessed_vectorized.pickle", "rb"))
#_, _, x_valid = pickle.load(open(paths.data + "/validation_preprocessed.pickle", "rb"))
y_test, x_tb, x_test = pickle.load(open("dbpedia_csv/" + "test_lem__preprocessed_vectorized.pickle", "rb"))

In [15]:
x_train[0]

'abbott farnham e abbott limited british coachbuilding business based farnham surrey trading name 1929 major part output subcontract motor vehicle manufacturer business closed 1972'

In [18]:
x_train[5]

'unsigned guide online contact directory career guide uk music industry founded 2003 first published printed directory unsigned guide became online resource november 2011'

In [22]:
len(x_train[0])

179

In [20]:



# x_train -> content
# y_train -> labels

# y_test -> labels
# x_test -> content

In [26]:
t = time()  # not compulsory

# loading CountVectorizer
tf_vectorizer = TfidfVectorizer() # or term frequency

X_train_tf = tf_vectorizer.fit_transform(x_train)

duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

Time taken to extract features from training data : 51.339539 seconds
n_samples: 560000, n_features: 682477


In [27]:
t = time()
X_test_tf = tf_vectorizer.transform(x_test)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

Time taken to extract features from test data : 7.214369 seconds
n_samples: 70000, n_features: 682477


In [28]:
t = time()

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, y_train)

training_time = time() - t
print("train time: %0.3fs" % training_time)

train time: 3.293s


In [29]:
t = time()
y_pred = naive_bayes_classifier.predict(X_test_tf)

test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred, target_names=[str(x) for x in range(14)]))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

test time:  0.189s
accuracy:   0.945
              precision    recall  f1-score   support

           0       0.92      0.85      0.88      5000
           1       0.94      0.97      0.96      5000
           2       0.93      0.76      0.84      5000
           3       0.98      0.99      0.98      5000
           4       0.96      0.96      0.96      5000
           5       0.96      0.99      0.97      5000
           6       0.94      0.95      0.94      5000
           7       0.97      0.98      0.98      5000
           8       1.00      0.95      0.97      5000
           9       1.00      0.98      0.99      5000
          10       0.99      0.99      0.99      5000
          11       0.88      0.99      0.93      5000
          12       0.93      0.96      0.94      5000
          13       0.85      0.92      0.89      5000

    accuracy                           0.94     70000
   macro avg       0.95      0.94      0.94     70000
weighted avg       0.95      0.94      0.94